<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_ELECTRICITY_Step4%20-%20combined%20MSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#-----------for whole dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data_V3.csv')


#-----------for test dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data_V3.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------



plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)


fx_window1 = 6
fx_window2 = 7
fx_window3 = 8


pred_step = 1

############################METHOD 1---------------- GO BY AIC ---------------------------------------------------------------------------------

aic1_count = list()
aic2_count = list()
aic3_count = list()
aic4_count = list()
aic5_count = list()
aic6_count = list()





for i in range(x.shape[0]-1) :

  x1 = x[i]
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results1 = regressor.fit(fx_window1)
  results2 = regressor.fit(fx_window2)
  results3 = regressor.fit(fx_window3)
  #results4 = regressor.fit(fx_window4)
  #results5 = regressor.fit(fx_window5)
  results6 = regressor.fit(int(y_pred[i]))

#Calculate AIC for ach case--------------
  try:
    aic1 = results1.aic
  except:
    aic1 = 9999
  aic1_count.append(aic1)

  try:
    aic2 = results2.aic
  except:
    aic2 = 9999
  aic2_count.append(aic2)

  try:
    aic3 = results3.aic
  except:
    aic3 = 9999
  aic3_count.append(aic3)

  #try:
   # aic4 = results4.aic
  #except:
   # aic4 = 9999
  #aic4_count.append(aic4)

  #try:
  #  aic5 = results5.aic
  #except:
  #  aic5 = 9999
  #aic5_count.append(aic5)


  try:
    aic6 = results6.aic
  except:
    aic6 = 9999
  aic6_count.append(aic6)


print("aic1_count : ", aic1_count)
print("aic2_count : ", aic2_count)
print("aic3_count : ", aic3_count)
#print("aic4_count : ", aic4_count)
#print("aic5_count : ", aic5_count)
print("aic6_count : ", aic6_count)

df = pd.DataFrame({'AIC1' : aic1_count, 'AIC2' : aic2_count, 'AIC3' : aic3_count,   'AIC6 - Predicted' : aic6_count})
df[0:1000].plot(figsize=(15,6))
plt.legend()
plt.show()

df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/AIC_comparison.csv', index = False)

################---------------------------METHOD 2- -GO BY RMSE----------------------------------------------------####################

def forecast_accuracy(forecast, actual):
    se = (forecast - actual)**2  # SE
    return se

#------------------------------------V1----------------------------------------

se_f1 = list()
se_f2 = list()
se_f3 = list()
se_fx = list()


sample_size = 1000
for i in range(sample_size) :
  x1 = x[i]
  df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6'])
  df_train, df_test = df[0:-pred_step], df[-pred_step:]
  model= VAR(df_train)

  model_fitted1 = model.fit(fx_window1)
  lag_order1 = model_fitted1.k_ar
  forecast_input1 = df.values[-lag_order1:]
  fc1 = model_fitted1.forecast(y=forecast_input1, steps=pred_step)
  df_forecast1 = pd.DataFrame(fc1, index=df.index[-pred_step:], columns=df.columns)
  se1 = forecast_accuracy(df_forecast1['V1'].values, df_test['V1'])
  se2 = forecast_accuracy(df_forecast1['V2'].values, df_test['V2'])
  se3 = forecast_accuracy(df_forecast1['V3'].values, df_test['V3'])
  se4 = forecast_accuracy(df_forecast1['V4'].values, df_test['V4'])
  se5 = forecast_accuracy(df_forecast1['V5'].values, df_test['V5'])
  se6 = forecast_accuracy(df_forecast1['V6'].values, df_test['V6'])
  se_f1.append(se1+se2+se3+se4+se5+se6)

  model_fitted2 = model.fit(fx_window2)
  lag_order2 = model_fitted2.k_ar
  forecast_input2 = df.values[-lag_order2:]
  fc2 = model_fitted2.forecast(y=forecast_input2, steps=pred_step)
  df_forecast2 = pd.DataFrame(fc2, index=df.index[-pred_step:], columns=df.columns)
  se1 = forecast_accuracy(df_forecast2['V1'].values, df_test['V1'])
  se2 = forecast_accuracy(df_forecast2['V2'].values, df_test['V2'])
  se3 = forecast_accuracy(df_forecast2['V3'].values, df_test['V3'])
  se4 = forecast_accuracy(df_forecast2['V4'].values, df_test['V4'])
  se5 = forecast_accuracy(df_forecast2['V5'].values, df_test['V5'])
  se6 = forecast_accuracy(df_forecast2['V6'].values, df_test['V6'])
  se_f2.append(se1+se2+se3+se4+se5+se6)

  model_fitted3 = model.fit(fx_window3)
  lag_order3 = model_fitted3.k_ar
  forecast_input3 = df.values[-lag_order3:]
  fc3 = model_fitted3.forecast(y=forecast_input3, steps=pred_step)
  df_forecast3 = pd.DataFrame(fc3, index=df.index[-pred_step:], columns=df.columns)
  se1 = forecast_accuracy(df_forecast3['V1'].values, df_test['V1'])
  se2 = forecast_accuracy(df_forecast3['V2'].values, df_test['V2'])
  se3 = forecast_accuracy(df_forecast3['V3'].values, df_test['V3'])
  se4 = forecast_accuracy(df_forecast3['V4'].values, df_test['V4'])
  se5 = forecast_accuracy(df_forecast3['V5'].values, df_test['V5'])
  se6 = forecast_accuracy(df_forecast3['V6'].values, df_test['V6'])
  se_f3.append(se1+se2+se3+se4+se5+se6)

  model_fittedx = model.fit(int(y_pred[i]))
  lag_orderx = model_fittedx.k_ar
  forecast_inputx = df.values[-int(y_pred[i]):]
  fcx = model_fittedx.forecast(y=forecast_inputx, steps=pred_step)
  df_forecastx = pd.DataFrame(fcx, index=df.index[-pred_step:], columns=df.columns)
  se1 = forecast_accuracy(df_forecastx['V1'].values, df_test['V1'])
  se2 = forecast_accuracy(df_forecastx['V2'].values, df_test['V2'])
  se3 = forecast_accuracy(df_forecastx['V3'].values, df_test['V3'])
  se4 = forecast_accuracy(df_forecastx['V4'].values, df_test['V4'])
  se5 = forecast_accuracy(df_forecastx['V5'].values, df_test['V5'])
  se6 = forecast_accuracy(df_forecastx['V6'].values, df_test['V6'])
  se_fx.append(se1+se2+se3+se4+se5+se6)

overall_se1 = sum(se_f1)
overall_se2 = sum(se_f2)
overall_se3 = sum(se_f3)
overall_sex = sum(se_fx)

overall_mse1 = overall_se1/len(se_f1)
overall_mse2 = overall_se2/len(se_f2)
overall_mse3 = overall_se3/len(se_f3)
overall_msex = overall_sex/len(se_fx)

print("overall_mse1 : ", overall_mse1)
print("overall_mse2 : ", overall_mse2)
print("overall_mse3 : ", overall_mse3)
print("overall_msex : ", overall_msex)

print("overall_se1 : ", overall_se1)
print("overall_se2 : ", overall_se2)
print("overall_se3 : ", overall_se3)
print("overall_sex : ", overall_sex)


overall_mse1 :  25    3.122963e-08
26    2.316462e-08
27    1.775912e-08
28    1.400867e-08
29    1.159907e-08
dtype: float64
overall_mse2 :  25    3.104642e-08
26    2.295241e-08
27    1.741182e-08
28    1.368813e-08
29    1.143981e-08
dtype: float64
overall_mse3 :  25    3.091841e-08
26    2.280015e-08
27    1.741715e-08
28    1.377629e-08
29    1.162753e-08
dtype: float64
overall_msex :  25    3.123503e-08
26    2.313466e-08
27    1.773516e-08
28    1.405225e-08
29    1.167328e-08
dtype: float64
overall_se1 :  25    0.000031
26    0.000023
27    0.000018
28    0.000014
29    0.000012
dtype: float64
overall_se2 :  25    0.000031
26    0.000023
27    0.000017
28    0.000014
29    0.000011
dtype: float64
overall_se3 :  25    0.000031
26    0.000023
27    0.000017
28    0.000014
29    0.000012
dtype: float64
overall_sex :  25    0.000031
26    0.000023
27    0.000018
28    0.000014
29    0.000012
dtype: float64


In [ ]:
from google.colab import drive
drive.mount('/content/drive')